In [1]:
%load_ext autoreload
%autoreload 2

from experiment_util import default_training_settings
from model import MLP
from pipeline.train import train_model

In [2]:
from enc import IPE, PE
from experiment_util import ExperimentModel

dataset_multi = 'collins-truemulti'
dataset_single = 'collins-single'

ipe = IPE(d_in=2, n_freqs=20)
pe_hd = PE(d_in=2, n_freqs=10)
pe_ld = PE(d_in=2, n_freqs=4)

models_experiment_single = [
    ExperimentModel('IPE', 'ipe-exp', MLP(ipe.d_output), ipe, dataset_single),
    ExperimentModel('PE-HD', 'pe-hd-exp', MLP(pe_hd.d_output), pe_hd, dataset_single),
    ExperimentModel('PE-LD', 'pe-ld-exp', MLP(pe_ld.d_output), pe_ld, dataset_single),
]

models_experiment_multi = [
    ExperimentModel('IPE-multi', 'ipe-exp', MLP(ipe.d_output), ipe, dataset_multi),
    ExperimentModel('PE-multi-HD', 'pe-hd-exp', MLP(pe_hd.d_output), pe_hd, dataset_multi),
    ExperimentModel('PE-multi-LD', 'pe-ld-exp', MLP(pe_ld.d_output), pe_ld, dataset_multi),
]


In [3]:
def retrain_models(models):
    for model in models:
        train_model(model.model, model.encoding,
                    f'{model.training_set_artifact}:latest',
                    train_settings,
                    output_artifact=model.artifact_name,
                    run_name=f'train-{model.display_name}')

train_settings = default_training_settings()
train_settings.lr = 0.007
train_settings.batch_size = 2200
train_settings.n_epochs = 25

In [4]:
retrain = False
if retrain:
    retrain_models(models_experiment_multi)

In [ ]:
from model_evaluation import res_to_scale
from pipeline.evaluation import evaluate_model_across_scales

scale_lo = 0
scale_hi = res_to_scale(8)
n_scales = 10

for model in [
    models_experiment_multi[0],
    models_experiment_multi[1]
]:
    res = 512
    evaluate_model_across_scales(f'{model.artifact_name}:latest',
                                 scale_lo, scale_hi,
                                 count=n_scales,
                                 run_name=model.display_name,
                                 res=res,
                                 assume_constant=model.encoding.type == 'PE')